In [1]:
(do 
    (require `serpent.core)
    (require `serpent.pprint)
    (require `serpent.jupyter)
    (serpent.core/configure)
    (add-dependencies [org.mitre.tdp/ttfs-workflow "3.1.25" :exclusions [[net.sf.jsi/jsi]]])
    (set-cda-config :root "tdp:3.1"))

Could not find artifact commons-codec:commons-codec:pom:1.16-20210806.145637-57 in mitre-snapshots (https://dali.mitre.org/nexus/content/repositories/mitre-caasd-snapshots)
Retrieving commons-codec/commons-codec/1.16-SNAPSHOT/commons-codec-1.16-20210806.145637-57.pom (16k)
    from https://repository.apache.org/snapshots/
Retrieving org/mitre/caasd/caasd-project/0.1.3-SNAPSHOT/maven-metadata.xml (1k)
    from https://dali.mitre.org/nexus/content/repositories/mitre-caasd-snapshots/
Retrieving org/mitre/caasd/jhlabs/proj4-library/1.0.6.1-SNAPSHOT/maven-metadata.xml (1k)
    from https://dali.mitre.org/nexus/content/repositories/mitre-caasd-snapshots/


{:configuration
 #object[org.apache.hadoop.conf.Configuration 0x718f3d21 "\"Configuration: core-default.xml, core-site.xml, mapred-default.xml, mapred-site.xml, yarn-default.xml, yarn-site.xml, hdfs-default.xml, hdfs-site.xml\""],
 :meta-repository
 #object[org.mitre.cda.hbase.HBaseMetaRepository 0x1d487096 "\"HBaseMetaRepository[tdp:3.1-meta]\""],
 :hbase-repository
 #object[org.mitre.cda.core.CachingKeyDereferencer 0x32f5d8f0 "\"org.mitre.cda.core.CachingKeyDereferencer@32f5d8f0\""]}


In [2]:
(add-dependencies [org.mitre.tdp/ttfs-procedure "3.1.25" :exclusions [[net.sf.jsi/jsi]]])
(add-dependencies [org.mitre.tdp/tdp-infrastructure "3.1.25" :exclusions [[net.sf.jsi/jsi]]])

("com.google.guava:guava:23.0"
 "net.java.dev.jets3t:jets3t:0.9.4"
 "org.json:json:20140107"
 "org.apache.commons:commons-math3:3.4.1"
 "org.codehaus.mojo:animal-sniffer-annotations:1.14"
 "com.google.errorprone:error_prone_annotations:2.0.18"
 "org.codehaus.jackson:jackson-xc:1.9.13"
 "org.apache.commons:commons-math:2.2"
 "org.scala-lang:scala-reflect:2.11.8"
 "org.apache.httpcomponents:httpcore:4.4.1"
 "com.google.j2objc:j2objc-annotations:1.1"
 "commons-cli:commons-cli:1.2"
 "com.google.code.findbugs:jsr305:1.3.9"
 "org.objenesis:objenesis:2.1"
 "joda-time:joda-time:2.9.4"
 "com.jamesmurty.utils:java-xmlbuilder:1.1"
 "net.iharder:base64:2.3.8"
 "commons-logging:commons-logging:1.2"
 "commons-codec:commons-codec:1.9"
 "org.scala-lang:scala-library:2.11.8"
 ...)


In [3]:
(import org.mitre.caasd.ttfs.ArincTransition
        org.mitre.caasd.ttfs.ArincFix
        org.mitre.caasd.ttfs.ArincAirway
        org.mitre.caasd.ttfs.ArincAirport)

org.mitre.caasd.ttfs.ArincAirport


In [4]:
;; data is small we can just materialize an entire cycle
(def cyc 20210101)

(def transitions (load-data ArincTransition :dates cyc :only "CIFP"))
(def fixes (load-data ArincFix :dates cyc :only "CIFP"))
(def airways (load-data ArincAirway :dates cyc :only "CIFP"))
(def airports (load-data ArincAirport :dates cyc :only "CIFP"))

(str "Transitions: " (count transitions) " Fixes: " (count fixes) " Airways: " (count airways) " Airports: " (count airports))

"Transitions: 61338 Fixes: 67911 Airways: 1550 Airports: 13765"


### Helper Functions

In [5]:
;; Grab all the transitions
(defn null-or-equals [e v]
    (or (= nil v) (= e nil) (= e v)))

(defn proc-filter [pname & kwargs]
    (let [{:keys [airport source] :or {airport nil source nil} :as opts} kwargs]
        (fn [p] (and 
                    (null-or-equals airport ((comp :name :airport) p)) 
                    (null-or-equals pname (:procedure_name p))
                    (null-or-equals source (:navigation_source p))))))

(defn fix-filter [col]
    (fn [f] 
        (some #(= % ((comp :name :identification) f)) col)))

(defn airway-filter [awname & kwargs]
    (let [{:keys [entry exit area-code source]
           :or {entry nil
                exit nil
                area-code nil
                source nil}
           :as opts}
          kwargs]
        (fn [a] (and
                    (null-or-equals awname ((comp :name :identification) a))
                    (some #(null-or-equals entry ((comp :name :identification :path_terminator) %)) (:points a))
                    (some #(null-or-equals exit ((comp :name :identification :path_terminator) %)) (:points a))
                    (null-or-equals area-code ((comp :customer_area_code :arinc_record_info) a))
                    (null-or-equals source ((comp :source :arinc_record_info) a))))))

(defn peek? [ele f] (f ele) ele)

(defn cda-key->string [k] (.toString (.getPrimaryKey k)))

#'user/cda-key->string


### Airway Mocking

In [6]:
(def airwayy (->> airways (filter #(= "Q6" (:identifier %)))))

#'user/airwayy


In [7]:
(defn fixName [leg] (str ((comp :name :identification :path_terminator) leg)))
(defn fixLatLon [leg] (let [latitude ((comp :latitude :path_terminator) leg)
                            longitude ((comp :longitude :path_terminator) leg)]
                          (str " , " latitude " , " longitude)))
(defn toMockitoAirwayBody [airway]
    (let [body (reduce str (->> (:legs airway)
     (map #(str "Leg " (fixName %) "= TF( \"" (fixName %) "\"" (fixLatLon %) "); \n"))))]
    (str body " - primary_key " (cda-key->string (:primary_key airway)) "\n")))

#'user/toMockitoAirwayBody


In [8]:
(->> airwayy (map toMockitoAirwayBody))

("Leg TKA= TF( "TKA" , 62.29837777777777 , -150.10524999999998); 
Leg JOKAP= TF( "JOKAP" , 63.91291111111111 , -150.97479166666668); 
Leg KUTDE= TF( "KUTDE" , 66.32227499999999 , -152.48358055555553); 
Leg LACIL= TF( "LACIL" , 69.504975 , -155.00948055555557); 
Leg BRW= TF( "BRW" , 71.27342777777778 , -156.78811666666667); 
 - primary_key 4-2YeCFS_9AAAJZ
")


### Transition Mocking

#### Procedures By Airport

In [17]:
(def airport-transitions (->> transitions (filter #(= "KPHL" ((comp :name :airport) %)))))
(frequencies (map :procedure_name airport-transitions))

{"R27R" 3,
 "L17" 4,
 "I17-Z" 4,
 "H09LZ" 3,
 "R26" 5,
 "L27L" 3,
 "R35" 5,
 "PAATS3" 9,
 "BUNTS2" 3,
 "VCN9" 3,
 "I09L" 4,
 "BOJID2" 6,
 "L26" 3,
 "L09L" 4,
 "R17" 4,
 "R27L" 4,
 "SPUDS4" 6,
 "I27R" 4,
 "I09RZ" 3,
 "I27L" 3,
 "I26" 3,
 "L27R" 4,
 "L09RZ" 3,
 "R09RY" 4,
 "SLATT4" 3,
 "R09LY" 4,
 "H09RZ" 3,
 "JIIMS3" 8,
 "VOR-A" 3}


In [18]:
(def proc-trans (filter (proc-filter "JIIMS3" :airport "KPHL" :source "CIFP") transitions))
(str "total transitions: " (count proc-trans))

"total transitions: 8"


In [19]:
(clojure.string/join "," (map (comp str get-key :primary_key) proc-trans))

"4mIH87B6x_AADul,4mIH87B6x_AADuv,4mIH87B6x_AADuw,4mIH87B6x_AADux,4mIH87B6x_AADu0,4mIH87B6x_AADu1,4mIH87B6x_AADu4,4mIH87B6x_AADu5"


In [24]:
(map :primary_key (filter #(= "KPHL" ((comp :name :identification) %)) airports))

(Key<4pvX1Gw_G4AACXi>)


In [26]:
(first (load-data org.mitre.caasd.ttfs.NfdcAirport :dates cyc :columns [:primary_key [:icao_identifier = "KPHL"] :ref_point_latitude :ref_point_longitude]))

Record
{:primary_key Key<4Fk9t23D4DAAEpn>,
 :icao_identifier "KPHL",
 :ref_point_latitude 39.872084,
 :ref_point_longitude -75.240663}


#### Mocking That Business

In [22]:
(defn toMockitoTransitionBody [transition]
    (reduce str (->> (:legs transition)
     (map #(str "Leg " (fixName %) "=" (:leg_type %) " ( \"" (fixName %) "\"" (fixLatLon %) "); \n")))))

(defn toMockitoTransitionMethod [transition] 
    (let [tname (:transition_name transition)
          pname (:procedure_name transition)
          aname ((comp :name :airport) transition)]
        (str "private static Transition " (:transition_name transition) "(){\n"
         (toMockitoTransitionBody transition)
         "return transition(\"" tname "\",\"" pname "\",\"" aname "\",,,);\n}")))

(defn mockitizeMeCaptain [transitions]
    (reduce str (->> transitions (map #(str (toMockitoTransitionMethod %) "\n")))))

#'user/mockitizeMeCaptain


In [23]:
(mockitizeMeCaptain proc-trans)

"private static Transition SWL(){
Leg SWL=IF ( "SWL" , 38.05659444444444 , -75.46390000000001); 
Leg RADDS=TF ( "RADDS" , 38.648555555555554 , -75.08846666666666); 
Leg WNSTN=TF ( "WNSTN" , 39.09550277777778 , -74.80033333333333); 
Leg HEKMN=TF ( "HEKMN" , 39.377075 , -74.90637222222223); 
Leg JIIMS=TF ( "JIIMS" , 39.53767222222222 , -74.96714444444444); 
return transition("SWL","JIIMS3","KPHL",,,);
}
private static Transition RW17(){
Leg JIIMS=IF ( "JIIMS" , 39.53767222222222 , -74.96714444444444); 
Leg SNEDE=TF ( "SNEDE" , 39.56268055555555 , -75.02313611111111); 
Leg SNEDE=FM ( "SNEDE" , 39.56268055555555 , -75.02313611111111); 
return transition("RW17","JIIMS3","KPHL",,,);
}
private static Transition RW09B(){
Leg JIIMS=IF ( "JIIMS" , 39.53767222222222 , -74.96714444444444); 
Leg TLUKS=TF ( "TLUKS" , 39.56359444444444 , -75.30269166666666); 
Leg CHEAZ=TF ( "CHEAZ" , 39.738327777777776 , -75.37936388888889); 
Leg STAYK=TF ( "STAYK" , 39.70346388888889 , -75.538575); 
Leg STAYK=FM ( "